<a href="https://colab.research.google.com/github/Shkum/IT_STEP/blob/ML/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%97_7_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [3]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [4]:
df = df[['Rent', 'Size', 'Furnishing Status', 'City']]
df.head()

,Rent,Size,Furnishing Status,City
0,10000.0,1100.0,Unfurnished,Kolkata
1,20000.0,800.0,Semi-Furnished,Kolkata
2,17000.0,1000.0,Semi-Furnished,Kolkata
3,10000.0,800.0,Unfurnished,Kolkata
4,7500.0,850.0,Unfurnished,Kolkata


# Завдання 1
Очистіть дані від викидів, розділіть на тренувальну та тестову чатини

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4746 entries, 0 to 4745
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rent               4369 non-null   float64
 1   Size               4402 non-null   float64
 2   Furnishing Status  4395 non-null   object 
 3   City               4388 non-null   object 
dtypes: float64(2), object(2)
memory usage: 314.4+ KB


In [6]:
df.describe()

,Rent,Size
count,4.369000e+03,4402.000000
mean,3.461306e+04,967.214675
std,7.740818e+04,636.118893
min,1.200000e+03,10.000000
25%,1.000000e+04,550.000000
50%,1.600000e+04,850.000000
75%,3.300000e+04,1200.000000
max,3.500000e+06,8000.000000


In [7]:
def remove_outliers(df, column_name):
  column = df[column_name]

  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)

  iqr = q3 - q1

  lower = q1 - 1.5*iqr
  upper = q3 + 1.5*iqr

  mask = (column >= lower) & (column <= upper)

  return df[mask]

for column_name in df.columns:
  if df.dtypes[column_name] in ("object", "datetime64[ns]"):
    continue

  df = remove_outliers(df, column_name)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 4745
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rent               3500 non-null   float64
 1   Size               3500 non-null   float64
 2   Furnishing Status  3244 non-null   object 
 3   City               3237 non-null   object 
dtypes: float64(2), object(2)
memory usage: 136.7+ KB


In [9]:
y = df['Rent']
X = df.drop(columns='Rent')

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression

In [12]:
num_columns = X.select_dtypes(include="number").columns

In [13]:
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [14]:
cat_columns = X.select_dtypes(include="object").columns

In [15]:
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())
           ])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['Size'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['Furnishing Status', 'City'], dtype='object'))])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [17]:
preprocessor.fit(X_train)
preprocessor.transform(X_train)

array([[7.50e+02, 0.00e+00, 1.00e+00, ..., 0.00e+00, 1.00e+00, 0.00e+00],
       [4.50e+02, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [4.50e+02, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       ...,
       [1.10e+03, 0.00e+00, 1.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [8.88e+02, 0.00e+00, 1.00e+00, ..., 0.00e+00, 0.00e+00, 1.00e+00],
       [1.15e+03, 0.00e+00, 1.00e+00, ..., 1.00e+00, 0.00e+00, 0.00e+00]])

In [18]:
model = Pipeline(
    [("prep", preprocessor),
     ("model", LinearRegression())
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

In [19]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Size'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

# Завдання 6
Виведіть основні метрики моделі

In [20]:
y_pred = model.predict(X_test)

In [21]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  # df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [22]:
all_metrics = pd.DataFrame()

In [23]:
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)
all_metrics["mean"] = get_metrics(y_test, np.full_like(y_test, y_test.mean()), "mean")

In [24]:
all_metrics

,train,test,mean
MAE,6417.22,6628.66,1.023726e+04
MSE,82871447.84,86714290.70,1.853827e+08
R2,0.53,0.53,0.000000e+00


# Завдання 7
Збережіть модель, завантажте її та спрогнозуйте ціну автомобіля

In [25]:
import joblib

In [26]:
joblib.dump(model, 'my_model')

['my_model']

In [27]:
my_model = joblib.load('my_model')

In [28]:
data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

data

,Size,Furnishing Status,City
0,900,Semi-Furnished,Mumbai


In [29]:
pred = my_model.predict(data)

In [30]:
pred

array([40380.85014142])

In [31]:
sample = pd.DataFrame({'Size': [np.nan], 'Furnishing Status': [np.nan], 'City': [np.nan]})
sample

,Size,Furnishing Status,City
0,NaN,NaN,NaN


In [32]:
pred = my_model.predict(sample)
pred

array([15403.75188306])